In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
CREATE TABLE ProfileCorrelation (
  SEQN1 VARCHAR(8),
  profile1 VARCHAR(18),
  SEQN2 VARCHAR(8),
  profile2 VARCHAR(18)
) AS SELECT
  SEQN1, profile1, SEQN2, profile2
FROM CSVREAD('../data/nhanes2005-2006/profile-pair-correlation.csv');

In [3]:
DROP VIEW IF EXISTS ProfileCorrelationNWeight;
DROP VIEW IF EXISTS ProfileCorrelationUnique;

CREATE VIEW ProfileCorrelationUnique AS
  SELECT DISTINCT * FROM ProfileCorrelation;

CREATE VIEW ProfileCorrelationNWeight AS
  SELECT PC.profile1 AS source, PC.profile2 as target, COUNT(*) as weight
  FROM ProfileCorrelationUnique PC
  GROUP BY PC.profile1, PC.profile2;
  
SELECT COUNT(*), SUM(weight) FROM ProfileCorrelationNWeight;
SELECT * FROM ProfileCorrelationNWeight;

In [4]:
CREATE VIEW ProfileCorrNWeight AS
SELECT source, target, weight w FROM ProfileCorrelationNWeight WHERE source < target
UNION
SELECT target, source, weight w FROM ProfileCorrelationNWeight WHERE source > target;

CREATE VIEW ProfileCorrFinalNWeight AS
SELECT source, target, SUM(w) AS weight
FROM ProfileCorrNWeight
GROUP BY source, target;

SELECT * FROM ProfileCorrFinalNWeight;

-- Gravação de pares de perfis com similaridade para rede
CALL CSVWRITE('../data/nhanes2005-2006/profile-pair-correlation-number.csv', 'SELECT * FROM ProfileCorrFinalNWeight');

In [5]:
CREATE VIEW ProfileCorrelationSWeight AS
  SELECT PC.profile1 AS source, PC.profile2 as target, COUNT(*) as weight
  FROM ProfileCorrelation PC
  GROUP BY PC.profile1, PC.profile2;
  
SELECT COUNT(*), SUM(weight) FROM ProfileCorrelationSWeight;
SELECT * FROM ProfileCorrelationSWeight;

-- Gravação de pares de perfis com similaridade para rede
CALL CSVWRITE('../data/nhanes2005-2006/profile-pair-correlation-similarity.csv', 'SELECT * FROM ProfileCorrelationSWeight');